<a href="https://colab.research.google.com/github/Tahimi/TriangulationBestPathModel/blob/main/triangulacaoSimples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from PIL import Image, ImageOps
def imageSizeInPixels(fileName):
    with Image.open(fileName) as image:
        return np.array(image.size) # [width, height]

In [ ]:
problemParams = {'backgroungImageFileName':'adim.jpg'}
print(imageSizeInPixels(problemParams['backgroungImageFileName']))

In [ ]:
#pip install utm
import utm
def GEEPositionInMetersFromLatitudeLongitude(latitudeLongitudeCoordinates):
    latitude = latitudeLongitudeCoordinates[0]; longitude = latitudeLongitudeCoordinates[1]
    xh, xv, zone, ut = utm.from_latlon(latitude,longitude)
    return np.array([xh,xv])

In [ ]:
GEEPositionInMetersFromLatitudeLongitude([1,1])

In [ ]:
problemParams.update({'ElevationImage':{'fileName':'test_2.png',
                                        'pixelSizeInMeters':np.array([30,30]),
                                        'latitudeLongitude':[1.,1.]},
                      'pixelsInCell':[1,1]})

def gridParamsFromImage():
    imageParams = problemParams['ElevationImage']

    imageSizeInPixels_ = imageSizeInPixels(imageParams['fileName'])
    gridSizeInPixels = imageSizeInPixels_
    gridSizeInMeters = gridSizeInPixels * imageParams['pixelSizeInMeters']
    gridSizeInIdx = (gridSizeInPixels / np.array(problemParams['pixelsInCell'])).astype(int)
    cellSizeInMeters = gridSizeInMeters / gridSizeInIdx

    gridParams = {'latitudeLongitude':imageParams['latitudeLongitude'],
                   'sizeInPixels':gridSizeInPixels,
                   'sizeInMeters':gridSizeInMeters,
                   'sizeInIdx':gridSizeInIdx,
                   'Cell':{'sizeInPixels':problemParams['pixelsInCell'],
                           'sizeInMeters':cellSizeInMeters}}
    gridParams.update({'GEEPositionInMeters':GEEPositionInMetersFromLatitudeLongitude(gridParams['latitudeLongitude'])})

    return gridParams

In [ ]:
import pprint
gridParams = gridParamsFromImage()
pprint.pprint(gridParams)

In [ ]:
def updateGridParamsForDisplay():
    from screeninfo import get_monitors
    screen = get_monitors()[0]
    screenOccupationCoeff = 0.9

    windowMaxOcupationInScreenPixels = np.array([screen.width, screen.height]) * screenOccupationCoeff
    #print("windowMaxOcupationInScreenPixels = ", windowMaxOcupationInScreenPixels)

    screenPixelsInMeter = min(windowMaxOcupationInScreenPixels / gridParams['sizeInMeters'])
    #print("screenPixelsInMeter = ", screenPixelsInMeter)

    cellSizeInScreenPixels = (screenPixelsInMeter * gridParams['Cell']['sizeInMeters']).astype(int)
    #print("cellSizeInScreenPixels = ", cellSizeInScreenPixels)

    windowSizeInScreenPixels = cellSizeInScreenPixels * gridParams['sizeInIdx']
    #print("windowSizeInScreenPixels = ", windowSizeInScreenPixels)
    
    gridParams.update({'sizeInScreenPixels':windowSizeInScreenPixels})
    gridParams['Cell'].update({'sizeInScreenPixels':cellSizeInScreenPixels})

In [ ]:
updateGridParamsForDisplay()
pprint.pprint(gridParams)

In [ ]:
class Cell:
    def __init__(self, Idx, grid):
        self.Idx = np.array(Idx) # [columnIdx, lineIdx]
        self.grid = grid
        self.data = {'elevation': 0.}
        self.centerIdx = self.Idx + np.array([1/2.,1/2.])

    def __eq__(self, other):
        return np.array_equal(self.Idx, other.Idx)

    def elevation(self):
        return self.data['elevation']

    def columnIdx(self):
        return self.Idx[0]

    def lineIdx(self):
        return self.Idx[1]

    def positionInScreenPixels(self):
        return self.grid.positionInScreenPixelsFromIdx(self.Idx)
    
    def positionInMeters(self):
        return self.grid.positionInMetersFromIdx(self.Idx)

    def centerPositionInScreenPixels(self):
        return self.grid.positionInScreenPixelsFromIdx(self.centerIdx)
    
    def centerPositionInMeters(self):
        return self.grid.positionInMetersFromIdx(self.centerIdx)

In [ ]:
class Grid:
    def __init__(self, gridParams):
        self.gridParams = gridParams

        self.cellSizeInMeters = self.gridParams['Cell']['sizeInMeters']
        self.cellSizeInScreenPixels = self.gridParams['Cell']['sizeInScreenPixels']
        self.sizeInMeters = self.gridParams['sizeInMeters']
        self.sizeInScreenPixels = self.gridParams['sizeInScreenPixels']
        self.GEEPositionInMeters = self.gridParams['GEEPositionInMeters']
        self.dim = self.gridParams['sizeInIdx'] # [columnsNbr, linesNbr]

        self.makeGridCells_()
        self.elevationIsSet = False

    def makeGridCells_(self):
        self.cells = np.ndarray(shape = self.dim, dtype=Cell)
        for j in range(self.columnsNbr()):
            for i in range(self.linesNbr()):
                self.cells[j,i] = Cell([j,i], self)

    def columnsNbr(self):
        return self.dim[0] # 41

    def linesNbr(self):
        return self.dim[1] # 18

    def cellsNbr(self):
        return self.columnsNbr() * self.linesNbr()

    def columnFromCell(self, cell):
        columnIdx = cell.columnIdx()
        return self.cells[columnIdx,:] # 18 element

    def lineFromCell(self, cell):
        lineIdx = cell.lineIdx()
        return self.cells[:,lineIdx] # 41 element

    def cellFromIdx(self, idx):
        columnIdx = int(idx[0]); lineIdx = int(idx[1])
        return self.cells[columnIdx, lineIdx]

    def positionInScreenPixelsFromIdx(self, idx):
        return (idx * self.cellSizeInScreenPixels).astype(int)
    
    def positionInMetersFromIdx(self, idx):
        positionIn2D = idx * self.cellSizeInMeters
        return np.append(positionIn2D, self.cellFromIdx(idx).elevation())

    def idxFromPositionInMeters(self, positionInMeters):
        positionIn2D = positionInMeters[:2]
        return positionIn2D / self.cellSizeInMeters

    def idxFromPositionInScreenPixels(self, positionInScreenPixels):
        return positionInScreenPixels / self.cellSizeInScreenPixels
    
    def cellFromPositionInMeters(self, positionInMeters):
        idx = self.idxFromPositionInMeters(positionInMeters)
        return self.cellFromIdx(idx)
    
    def cellFromPositionInScreenPixels(self, positionInScreenPixels):
        idx = self.idxFromPositionInScreenPixels(positionInScreenPixels)
        return self.cellFromIdx(idx)

    '''def cellAtLatitudeLongitude(self, latitudeLongitude):
        GEEPositionInMetersOfCell = GEEPositionInMetersFromLatitudeLongitude(latitudeLongitude)
        positionInMeters2D = GEEPositionInMetersOfCell - self.GEEPositionInMeters
        return self.cellAtPositionInMeters(positionInMeters2D)

    def latitudeLongitudeOfCell(self, cell):
        GEEPositionInMetersOfCell = self.GEEPositionInMeters + cell.centerPositionInMeters2D()'''
    
    def cellsIdx(self, cells):
        listOfIdxs = []
        for cell in cells:
            listOfIdxs.append(cell.Idx)
        return listOfIdxs

    def setElevation(self):
        for j in range(self.columnsNbr()):
            for i in range(self.linesNbr()):
                self.cells[j,i].data['elevation'] = elevation[j,i]
        self.elevationIsSet = True

In [ ]:
grid = Grid(gridParams)

In [ ]:
print(grid.dim)

In [ ]:
A = grid.cellFromIdx([2,3])
print("A.columnIdx() = ", A.columnIdx())
print("A.lineIdx() = ", A.lineIdx())

In [ ]:
for cell in grid.columnFromCell(A):
    print("cell.Idx = ", cell.Idx)

In [ ]:
leftSemiColumn = grid.columnFromCell(A)[:A.lineIdx()]
for cell in np.flip(leftSemiColumn):
    print("cell.Idx = ", cell.Idx)

In [ ]:
def dataFromImage(grid, fileName):
    data = None
    with Image.open(fileName) as image:
        np.savetxt(fileName+".txt", np.matrix(image, np.float64).T, fmt="%3.f", delimiter=",")
        if not np.array_equal(grid.dim, image.size):
            image = image.resize(grid.dim)
        data = np.matrix(image, np.float64).T
    return data

In [ ]:
elevation = dataFromImage(grid, 'test_2.png')
print("grid.dim = ", grid.dim, ", elevation.shape = ", elevation.shape)

In [ ]:
print("elevation.shape = ", elevation.shape)
print("elevation[0,1] = ", elevation[0,1])

In [ ]:
grid.setElevation()

In [ ]:
import os, sys
print(os.getcwd())
sys.path.insert(0, os.getcwd())

In [ ]:
# pip install pygame
import pygame as pg
from pygame.locals import *
#import threading
#import time

class DisplayManager:
    def __init__(self, grid, windowCaptionParte1 = f'Triangulacao'):
        self.grid = grid
        self.windowCaptionParte1 = windowCaptionParte1

        self.windowDim = self.grid.gridParams['sizeInScreenPixels']
        self.windowWidth = self.windowDim[0]
        self.windowHeight = self.windowDim[1]

        self.A = None; self.AIsSet = False
        self.B = None; self.BIsSet = False
        self.solver = None; self.solverIsSet = False
        self.updateDataSpan_ = None
        
        self.backgroundImage = pg.image.load(problemParams['backgroungImageFileName'])
        self.backgroundImage = pg.transform.scale(self.backgroundImage, self.windowDim)
        self.initiateDisplay()
        
    def initiateDisplay(self):
        pg.init()
        pg.font.init()
        
        self.window = pg.display.set_mode(self.windowDim)
        pg.display.set_caption(self.windowCaptionParte1)
        self.window.blit(self.backgroundImage, (0,0))

        self.clock = pg.time.Clock()
        self.clock.tick()
        self.display = True
        self.updating = True
        self.iter = -1

    def problemIsSet(self):
        return self.grid.elevationIsSet and self.AIsSet and self.BIsSet

    def setAFromIdx(self, idx):
        self.A = grid.cellFromIdx(idx)
        self.grid.A = self.A
        self.AIsSet = True
        print("'A' cell selected, ", idx)

    def setBFromIdx(self, idx):
        self.B = grid.cellFromIdx(idx)
        self.grid.B = self.B
        self.BIsSet = True
        print("'B' cell selected, ", idx)

    def setSolver(self, solver):
        self.solver = solver
        self.solverIsSet = True

    def update(self):
        self.draw()
        #self.printData()
        pg.display.set_caption(self.windowCaption())
        pg.display.flip()
        self.processEvent()
        
    def draw(self):
        if self.problemIsSet():
            self.drawElevationField()
            self.drawCell(self.A, pg.Color('green'), True)
            self.drawCell(self.B, pg.Color('red'), True)
            self.drawLine(pg.Color('blue'), self.A, self.B)

        if self.solverIsSet:
            self.drawLines(pg.Color('darkmagenta'), self.solver.bestPathCells)

        self.drawGrid()

    def printData(self):
        if not self.grid.elevationIsSet: return
        self.printElevationField()
        
    def drawGrid(self):
        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                self.drawCell(cell)

    def drawElevationField(self, minColor = pg.Color('black'), maxColor = pg.Color('white')):
        if not self.grid.elevationIsSet: return
        self.updateDataSpan()

        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                dataValue = cell.elevation()
                if self.spanOfData != 0:
                    coeff = (dataValue - self.minExposedDataValue) / self.spanOfData
                    color = minColor.lerp(maxColor, coeff)
                else:
                    color = pg.Color('gray85')

                self.drawCell(cell, color, True)
        
    def updateDataSpan(self):
        if self.updateDataSpan_ is None:
            self.minExposedDataValue = +np.inf
            self.maxExposedDataValue = -np.inf

            for j in range(self.grid.columnsNbr()):
                for i in range(self.grid.linesNbr()):
                    cell = self.grid.cellFromIdx([j,i])

                    if cell.elevation() < self.minExposedDataValue: self.minExposedDataValue = cell.elevation()
                    if cell.elevation() > self.maxExposedDataValue: self.maxExposedDataValue = cell.elevation()

            self.spanOfData = self.maxExposedDataValue - self.minExposedDataValue
        return self.updateDataSpan_

    def drawCells(self, cells, color = pg.Color('cadetblue1'), fillCell = True):
        for cell in cells:
            self.drawCell(cell, color, fillCell)

        pg.display.flip()
        self.processEvent()

    def drawCell(self, cell, color = pg.Color('black'), fillCell = False):
        if fillCell:
            pg.draw.rect(self.window, color, pg.Rect(cell.positionInScreenPixels(), self.grid.cellSizeInScreenPixels))
        else:
            pg.draw.rect(self.window, color, pg.Rect(cell.positionInScreenPixels(), self.grid.cellSizeInScreenPixels), 1)

    def drawLine(self, color, A, B):
        pg.draw.line(self.window, color, A.centerPositionInScreenPixels(), B.centerPositionInScreenPixels(), 2)

    def drawLines(self, color, cells):
        if len(cells) < 2: return

        for i in range(1, len(cells)):
            self.drawLine(color, cells[i - 1], cells[i])

    def printElevationField(self):
        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                dataValueSTR = str(int(cell.elevation()))
                self.printCellData(cell, dataValueSTR)

    def printCellData(self, cell, dataSTR, board_thikness=1):
        font = pg.font.SysFont('timesnewroman', 12)
        text = font.render(dataSTR, True, pg.Color('black'), pg.Color('grey99'))
        textRect = text.get_rect()
        textRect.center = cell.centerPositionInScreenPixels()
        self.window.blit(text, textRect)

    def processEvent(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                self.display = False

            elif event.type == pg.KEYDOWN:
                if event.key == pg.K_SPACE:
                    self.updating = False

        while not self.updating: self.handSelection()
        if not self.display: pg.quit()

    def handSelection(self):
        for evnt in pg.event.get():
            if evnt.type == pg.QUIT:
                self.display = False
                self.updating = True

            if evnt.type == pg.MOUSEBUTTONDOWN:
                cell = self.grid.cellFromPositionInScreenPixels(pg.mouse.get_pos())

                if evnt.button == 1:
                    self.setAFromIdx(cell.Idx)

                if evnt.button == 3:
                    self.setBFromIdx(cell.Idx)

            elif evnt.type == pg.KEYDOWN:
                if evnt.key == pg.K_SPACE:
                    self.updating = True

    def windowCaption(self):
        self.iter = self.iter + 1
        return self.windowCaptionParte1 + ", iter: " + str(self.iter)

In [ ]:
displayManager = DisplayManager(grid)

In [ ]:
displayManager.initiateDisplay()
while displayManager.display:
    displayManager.update()

In [ ]:
print(displayManager.A.Idx)
print(displayManager.B.Idx)

In [ ]:
problemParams.update({'waterSourceIdx':displayManager.A.Idx, 'waterDestinationIdx':displayManager.B.Idx})
problemParams.update({'Q':1.})

In [ ]:
pprint.pprint(problemParams)

In [ ]:
import math
def norm2Length(p1, p2):
    dislocation = np.array(p2) - np.array(p1)
    return math.sqrt(np.dot(dislocation.T, dislocation))
        
class Segment:
    def __init__(self, grid, cI, cF):
        self.grid = grid
        self.cI = cI
        self.cF = cF
        self.norm2Length = norm2Length(self.cI.centerPositionInMeters(), self.cF.centerPositionInMeters())
        self.profil = None

    def length(self):
        if self.profil is None: self.profil = SegmentProfil(self, self.grid)
        return self.profil.cumulativeLength()

    def maxRelativeElevation(self):
        if self.profil is None: self.profil = SegmentProfil(self, self.grid)
        return self.profil.maxRelativeElevation()

In [ ]:
class SegmentProfil:
    def __init__(self, segment, grid):
        self.segment = segment
        self.grid = grid
        self.data = {'midPointsInMeters':[],
                     'relativeElevations':[],
                     'cumulativeLengths':[]}
        self.maxRelativeElevation_ = None
        self.process_()

    def midPointsInMeters(self):
        return self.data['midPointsInMeters']

    def relativeElevations(self):
        return self.data['relativeElevations']

    def maxRelativeElevation(self):
        if self.maxRelativeElevation_ is None:
            self.maxRelativeElevation_ = max(self.relativeElevations())
        return self.maxRelativeElevation_

    def cumulativeLengths(self):
        return self.data['cumulativeLengths']

    def cumulativeLength(self):
        return self.cumulativeLengths()[-1]

    def intersectedGridLinesIdxs_(self, A, B, axis):
        x0 = A[axis] #; print("x0 = ", x0)
        x1 = B[axis] #; print("x1 = ", x1)
        dx = x1-x0
        xMin = min(x0, x1)
        xMax = max(x0, x1)
        isign = int(np.sign(x1 - x0))
        v = [*range(int(xMin) + 1, int(xMax) + 1)] #; print("v = ", v)
        if isign == -1:
            v.reverse()
        return v

    def intersectionWithGridLines_(self, A, B, axis):
        v = []
        for i in self.intersectedGridLinesIdxs_(A, B, axis):
            ti = (i - A[axis]) / (B[axis] - A[axis]) #; print("ti = ", ti)
            xi = A[0] + ti * (B[0] - A[0]) #; print("xi = ", xi)
            yi = A[1] + ti * (B[1] - A[1]) #; print("yi = ", yi)
            v.append([ti, xi, yi])
        return v

    def process_(self):
        A = self.segment.cI.centerIdx #; print("A = ", A)
        B = self.segment.cF.centerIdx #; print("B = ", B)

        vi = self.intersectionWithGridLines_(A, B, 0) #; print("vi = ", vi)
        vj = self.intersectionWithGridLines_(A, B, 1) #; print("vj = ", vj)

        # # making the curve connecting intersection points [t, x, y]
        curve = [[0., A[0], A[1]]]
        curve.extend(vi)
        curve.extend(vj)
        curve.extend([[1., B[0], B[1]]])
        curve = np.array(curve)
        curve = curve[curve[:, 0].argsort()]
        curveInIdx = curve #; print("curveInIdx = ", curveInIdx)

        # deleting repeated points
        curve = [curveInIdx[0]]
        i = 1 
        while i < len(curveInIdx):
            if curveInIdx[i,0] - curveInIdx[i-1,0] < 10**-6:
                i = i+1
            curve.append(curveInIdx[i])
            i = i+1
        curveInIdx = curve #; print("curveInIdx = ", curveInIdx)

        # making the curve connecting intersection mid-points
        curve = [[0., A[0], A[1]]]
        for i in range(len(curveInIdx) - 1):
            curve.append((curveInIdx[i] + curveInIdx[i + 1]) / 2.)
        curve.append([1., B[0], B[1]])
        curveOfMidPointsInIdx = curve #; print("curveOfMidPointsInIdx = ", curveOfMidPointsInIdx)

        curve = []
        for point in curveOfMidPointsInIdx:
            curve.append(self.grid.positionInMetersFromIdx([point[1], point[2]]))
        self.data['midPointsInMeters'] = curve #; print("self.data['midPointsInMeters'] = ", self.data['midPointsInMeters'])

        lastPoint = self.data['midPointsInMeters'][0]
        cumulativeLength = 0.
        for point in self.data['midPointsInMeters']:
            self.data['relativeElevations'].append(point[2]-lastPoint[2])
            cumulativeLength = cumulativeLength + norm2Length(lastPoint, point)
            self.data['cumulativeLengths'].append(cumulativeLength)
            lastPoint = point

    def intersectedCells(self):
        cells = []
        for i in range(1, len(self.data['midPointsInMeters'])-1):
            cells.append(self.grid.cellFromPositionInMeters(self.data['midPointsInMeters'][i]))
        return cells

In [ ]:
AB = Segment(grid, grid.A, grid.B)

In [ ]:
if AB.profil is None: print("None")

In [ ]:
print("AB.norm2Length = ", AB.norm2Length)

In [ ]:
print("AB.length() = ", AB.length())

In [ ]:
AB.profil.midPointsInMeters()

In [ ]:
for cell in AB.profil.intersectedCells():
    print(cell.Idx)

In [ ]:
class Path:
    def __init__(self, grid, cellsIdx):
        self.grid = grid
        self.cellsIdx = cellsIdx
        
        self.cells = []
        for idx in self.cellsIdx:
            cell = self.grid.cellFromIdx(idx)
            self.cells.append(cell)

        self.segments = []
        for i in range(len(self.cells) - 1):
            segment = Segment(self.grid, self.cells[i], self.cells[i+1])
            self.segments.append(segment)

        # compute norm2Length
        self.norm2Length = 0.
        for segment in self.segments:
            self.norm2Length = self.norm2Length + segment.norm2Length

        self.length_ = None
        self.maxRelativeElevation_ = None

    def length(self):
        if self.length_ is None:
            # compute length_
            self.length_ = 0.
            for segment in self.segments:
                self.length_ = self.length_ + segment.length()
        return self.length_

    def maxRelativeElevation(self):
        if self.maxRelativeElevation_ is None:
            # compute maxRelativeElevation_
            self.maxRelativeElevation_ = 0.
            for segment in self.segments:
                self.maxRelativeElevation_ = max(self.maxRelativeElevation_, segment.maxRelativeElevation())
        return self.maxRelativeElevation_

In [ ]:
projectParams = {'implementationLinearCost':1., 'energyUnitCost':1.}
class CostManager:
    def __init__(self, projectParams):
        self.projectParams = projectParams

    def implementationCost(self, path):
        return path.length() * self.projectParams['implementationLinearCost']

    def energyCost(self, path):
        return path.maxRelativeElevation() * self.projectParams['energyUnitCost']

    def totalCost(self, path):
        return self.implementationCost(path) + self.energyCost(path)
    
    def roughCost(self, path):
        return path.norm2Length * self.projectParams['implementationLinearCost']

In [ ]:
ABPath = Path(grid, grid.cellsIdx([grid.A, grid.B]))

In [ ]:
print("ABPath.norm2Length = ", ABPath.norm2Length)
print("ABPath.length() = ", ABPath.length())

In [ ]:
costManager = CostManager(projectParams)
print("roughCost = ", costManager.roughCost(ABPath))
print("totalCost = ", costManager.totalCost(ABPath))

In [ ]:
class Solver:
    def __init__(self, displayManager):
        self.displayManager = displayManager
        self.displayManager.setSolver(self)
        self.grid = self.displayManager.grid
        self.A = self.displayManager.A
        self.B = self.displayManager.B
        self.costManager = CostManager(projectParams)

    def solve(self):
        minTotalCost = self.costManager.totalCost(Path(self.grid,[self.A.Idx,self.B.Idx]))
        self.selectedMinCosts = [minTotalCost]
        self.bestPathCells = [self.A]
        print(self.bestPathCells[-1].Idx, " ", "{:.6f}".format(self.selectedMinCosts[-1]))

        while True:
            self.updateCMs()
            minTotalCost = +np.inf
            for CM in self.CMs:
                #print('CM.Idx = ', CM.Idx)
                if CM == self.currentCell: continue
                twoSegmentsPath = Path(self.grid, [self.currentCell.Idx, CM.Idx, self.B.Idx])
                roughCost = self.costManager.roughCost(twoSegmentsPath)
                if self.costManager.roughCost(twoSegmentsPath) > minTotalCost: continue
                totalCost = self.costManager.totalCost(twoSegmentsPath)
                '''implementationCost = self.costManager.implementationCost(twoSegmentsPath)
                energyCost = self.costManager.energyCost(twoSegmentsPath)'''
                if totalCost < minTotalCost:
                    '''print("--> tubCost = ", "{:.4f}".format(implementationCost),
                          ", energyCost = ", "{:.4f}".format(energyCost),
                          ", totalCost = ", "{:.4f}".format(totalCost),
                          ", minTotalCost = ", "{:.4f}".format(minTotalCost))'''
                    minTotalCost = totalCost
                    selectedCM = CM

            self.bestPathCells.append(selectedCM)
            self.selectedMinCosts.append(minTotalCost)
            print(self.bestPathCells[-1].Idx, " ", "{:.6f}".format(self.selectedMinCosts[-1]))

            if self.bestPathCells[-1] == self.B: return
    
    def updateCMs(self):
        self.updateCMOfReference()
        
        A = self.CMOfReference
        column = grid.columnFromCell(A); line = grid.lineFromCell(A)
        columnIdx = A.columnIdx(); lineIdx = A.lineIdx()

        self.CMs = [A]
        leftSemiColumn = column[:lineIdx]; self.CMs.extend(leftSemiColumn)
        leftSemiLine = line[:columnIdx]; self.CMs.extend(leftSemiLine)
        rightSemiColumn = column[lineIdx+1:]; self.CMs.extend(rightSemiColumn)
        rightSemiLine = line[columnIdx+1:]; self.CMs.extend(rightSemiLine)
        
    def updateCMOfReference(self):
        self.currentCell = self.bestPathCells[-1]
        
        deltaJ = np.sign(self.B.columnIdx() - self.currentCell.columnIdx())
        nextColumnIdx = self.currentCell.columnIdx() + deltaJ
        
        deltaI = np.sign(self.B.lineIdx() - self.currentCell.lineIdx())
        nextLineIdx = self.currentCell.lineIdx() + deltaI
        
        self.CMOfReference = self.grid.cellFromIdx([nextColumnIdx, nextLineIdx])
        #print('self.CMOfReference.Idx = ' , self.CMOfReference.Idx)

    def totalCost(self, path):
        return self.costManager.totalCost(path)

    def bestPathTotalCost(self):
        return self.totalCost(self.bestPath())

    def bestPath(self):
        return Path(self.grid, self.grid.cellsIdx(self.bestPathCells))

In [ ]:
solver = Solver(displayManager)

In [ ]:
import timeit, time, datetime

print("problemParams = "); pprint.pprint(problemParams); print()
print("gridParams = "); pprint.pprint(gridParams); print()

print("Simulation start:Searching the best path from A to B...\n")
simStartInstant = timeit.default_timer()
solver.solve()
deltaT = timeit.default_timer() - simStartInstant
print("\nbestPathTotalCost = ", "{:.0f}".format(solver.bestPathTotalCost()))
print("The simulation took: ", datetime.timedelta(seconds = deltaT)," (hh:mm:ss)")

In [ ]:
for cell in solver.bestPathCells:
    print(cell.Idx)

In [ ]:
displayManager.initiateDisplay()
while displayManager.display:
    displayManager.update()

In [ ]:
solver.bestPathTotalCost()